In [ ]:
import os
import time
import random
import sqlite3
import requests

import numpy as np
from tqdm.notebook import tqdm
import pandas as pd
tqdm.pandas()

from langchain_openai import ChatOpenAI
from langchain_mcp_adapters.client import MultiServerMCPClient
from langchain.agents import create_agent

os.environ["OPENAI_API_KEY"] = "your key here"

In [2]:
process_adaptations_dict = { 
    "base_rule": {
        "J09A": "Use the generate_process_rule_from_bpmn tool to generate a process rule from the following BPMN file: bpmn/J09A_base_rule.bpmn and store it in the database as J09A",
        "J09B": "Use the generate_process_rule_from_bpmn tool to generate a process rule from the following BPMN file: bpmn/J09B_base_rule.bpmn and store it in the database as J09B",
        "J09C": "Use the generate_process_rule_from_bpmn tool to generate a process rule from the following BPMN file: bpmn/J09C_base_rule.bpmn and store it in the database as J09C",
        "J09D": "Use the generate_process_rule_from_bpmn tool to generate a process rule from the following BPMN file: bpmn/J09D_base_rule.bpmn and store it in the database as J09D",
    },
    "0_values": {
        "J09A": "Use the generate_process_rule_from_bpmn tool to generate a process rule from the following BPMN file: bpmn/J09A_0_values.bpmn and store it in the database as J09A",
        "J09B": "Use the generate_process_rule_from_bpmn tool to generate a process rule from the following BPMN file: bpmn/J09B_0_values.bpmn and store it in the database as J09B",
        "J09C": "Use the generate_process_rule_from_bpmn tool to generate a process rule from the following BPMN file: bpmn/J09C_0_values.bpmn and store it in the database as J09C",
        "J09D": "Use the generate_process_rule_from_bpmn tool to generate a process rule from the following BPMN file: bpmn/J09D_0_values.bpmn and store it in the database as J09D",
    },
    "500_values": {
        "J09A": "Use the generate_process_rule_from_bpmn tool to generate a process rule from the following BPMN file: bpmn/J09A_500_values.bpmn and store it in the database as J09A",
        "J09B": "Use the generate_process_rule_from_bpmn tool to generate a process rule from the following BPMN file: bpmn/J09B_500_values.bpmn and store it in the database as J09B",
        "J09C": "Use the generate_process_rule_from_bpmn tool to generate a process rule from the following BPMN file: bpmn/J09C_500_values.bpmn and store it in the database as J09C",
        "J09D": "Use the generate_process_rule_from_bpmn tool to generate a process rule from the following BPMN file: bpmn/J09D_500_values.bpmn and store it in the database as J09D",
    },
    "900_values": {
        "J09A": "Use the generate_process_rule_from_bpmn tool to generate a process rule from the following BPMN file: bpmn/J09A_900_values.bpmn and store it in the database as J09A",
        "J09B": "Use the generate_process_rule_from_bpmn tool to generate a process rule from the following BPMN file: bpmn/J09B_900_values.bpmn and store it in the database as J09B",
        "J09C": "Use the generate_process_rule_from_bpmn tool to generate a process rule from the following BPMN file: bpmn/J09C_900_values.bpmn and store it in the database as J09C",
        "J09D": "Use the generate_process_rule_from_bpmn tool to generate a process rule from the following BPMN file: bpmn/J09D_900_values.bpmn and store it in the database as J09D",
    },
    "city_values": {
        "J09A": "Use the generate_process_rule_from_bpmn tool to generate a process rule from the following BPMN file: bpmn/J09A_city_values.bpmn and store it in the database as J09A",
        "J09B": "Use the generate_process_rule_from_bpmn tool to generate a process rule from the following BPMN file: bpmn/J09B_city_values.bpmn and store it in the database as J09B",
        "J09C": "Use the generate_process_rule_from_bpmn tool to generate a process rule from the following BPMN file: bpmn/J09C_city_values.bpmn and store it in the database as J09C",
        "J09D": "Use the generate_process_rule_from_bpmn tool to generate a process rule from the following BPMN file: bpmn/J09D_city_values.bpmn and store it in the database as J09D",
    },
    "extension_estimates": {
        "J09A": "Use the generate_process_rule_from_bpmn tool to generate a process rule from the following BPMN file: bpmn/J09A_extension_estimates.bpmn and store it in the database as J09A",
        "J09B": "Use the generate_process_rule_from_bpmn tool to generate a process rule from the following BPMN file: bpmn/J09B_extension_estimates.bpmn and store it in the database as J09B",
        "J09C": "Use the generate_process_rule_from_bpmn tool to generate a process rule from the following BPMN file: bpmn/J09C_extension_estimates.bpmn and store it in the database as J09C",
        "J09D": "Use the generate_process_rule_from_bpmn tool to generate a process rule from the following BPMN file: bpmn/J09D_extension_estimates.bpmn and store it in the database as J09D",
    },
    "extension_mail": {
        "J09A": "Use the generate_process_rule_from_bpmn tool to generate a process rule from the following BPMN file: bpmn/J09A_extension_mail.bpmn and store it in the database as J09A",
        "J09B": "Use the generate_process_rule_from_bpmn tool to generate a process rule from the following BPMN file: bpmn/J09B_extension_mail.bpmn and store it in the database as J09B",
        "J09C": "Use the generate_process_rule_from_bpmn tool to generate a process rule from the following BPMN file: bpmn/J09C_extension_mail.bpmn and store it in the database as J09C",
        "J09D": "Use the generate_process_rule_from_bpmn tool to generate a process rule from the following BPMN file: bpmn/J09D_extension_mail.bpmn and store it in the database as J09D",
    },
}

In [3]:
random.seed(42)
SEEDS = [42,]
SEEDS += random.sample(range(0, 10000), 4)
print(SEEDS)
from itertools import product
process_adaptations = process_adaptations_dict.keys()
rule_adaptation_methods = ["generate_bpmn", ] # "generate_bpmn"
tours = ["J09A", "J09B", "J09C", "J09D"]
runs = list(product(process_adaptations, rule_adaptation_methods, tours, SEEDS))
runs = random.sample(runs, 25)
runs

[42, 1824, 409, 4506, 4012]


[('500_values', 'generate_bpmn', 'J09D', 409),
 ('0_values', 'generate_bpmn', 'J09D', 42),
 ('0_values', 'generate_bpmn', 'J09B', 1824),
 ('0_values', 'generate_bpmn', 'J09A', 409),
 ('extension_estimates', 'generate_bpmn', 'J09B', 4506),
 ('base_rule', 'generate_bpmn', 'J09B', 4506),
 ('base_rule', 'generate_bpmn', 'J09B', 409),
 ('0_values', 'generate_bpmn', 'J09A', 4506),
 ('500_values', 'generate_bpmn', 'J09D', 42),
 ('500_values', 'generate_bpmn', 'J09D', 4012),
 ('extension_mail', 'generate_bpmn', 'J09B', 4012),
 ('base_rule', 'generate_bpmn', 'J09B', 1824),
 ('500_values', 'generate_bpmn', 'J09C', 42),
 ('city_values', 'generate_bpmn', 'J09C', 1824),
 ('city_values', 'generate_bpmn', 'J09A', 4506),
 ('city_values', 'generate_bpmn', 'J09B', 4012),
 ('900_values', 'generate_bpmn', 'J09B', 4012),
 ('500_values', 'generate_bpmn', 'J09C', 4506),
 ('0_values', 'generate_bpmn', 'J09B', 4506),
 ('extension_mail', 'generate_bpmn', 'J09D', 4012),
 ('900_values', 'generate_bpmn', 'J09D', 4

In [4]:
for run in runs:
    process_adaptation, rule_adaptation_method, tour, seed = run
    
    session_id = f"processadaptation_{process_adaptation}_processadaptationmethod_{rule_adaptation_method}_tour_{tour}_seed_{seed}_exception_handling_db_error"
    if os.path.exists(f"sessions/{session_id}/"):
        print(f"Session {session_id} already exists. Skipping...")
        continue

    with open("seed.txt", "w", encoding="utf-8") as f:
        f.write(str(seed))
    
    BASE_FRAME_TEMPLATE = process_adaptations_dict.get(process_adaptation).get(tour)
    
    # Drop process_rules table to start fresh each time
    conn = sqlite3.connect('database.db')
    cursor = conn.cursor()
    cursor.execute("DROP TABLE IF EXISTS process_rules")
    conn.commit()
    conn.close()
    conn = sqlite3.connect('database.db')
    cursor = conn.cursor()
    cursor.execute("""
        CREATE TABLE process_rules (
            process_rule_id TEXT PRIMARY KEY,
            process_rule TEXT NOT NULL
        )
    """)
    conn.commit()
    conn.close()
    
    client = MultiServerMCPClient(  
        {
            "fastapi": {
                "transport": "streamable_http",  # HTTP-based remote server
                # Ensure you start your fastapi server on port 8000
                "url": "http://localhost:8001/mcp",
            }
        }
    )
    tools = await client.get_tools()

    SYSTEM_PROMPT = """You are a process-frame agent with three main capabilities. 
First, you can take a BPMN file path and generate a structured process rule from it, automatically storing it in the rule database along with an ID. 
Second, you can take a natural language description from a human and also convert that into a structured process rule, automatically writing it to the database with an ID. 
Third, if a user requests that a specific process ID be run, you can call a process-execution agent to handle that execution. 
Always follow these steps and ensure each rule is stored and each process is run as requested."""

    config = {
        "model": "gpt-5.1",
        #"service_tier": "priority",
        "temperature": 0,
        "seed": seed,
        "reasoning_effort": "high",
    }

    model = ChatOpenAI(
        **config
    )

    frame_agent = create_agent(
        model,
        tools=tools,
        system_prompt=SYSTEM_PROMPT,
    )
    
    # Add process rule to the database
    print(f"Adding process rule for session {session_id}...")
    rule_add_prompt = ""

    if rule_adaptation_method == "add":
        PROMPT_TEMPLATE = """**Add** this process rule with id '{tour}' to the rule database:

{BASE_FRAME_TEMPLATE_FILLED}"""
        prompt = PROMPT_TEMPLATE.format(tour=tour, BASE_FRAME_TEMPLATE_FILLED=BASE_FRAME_TEMPLATE.format(tour=tour))
    elif rule_adaptation_method == "generate_bpmn":
        prompt = BASE_FRAME_TEMPLATE
    with open(f"session_id.txt", "w", encoding="utf-8") as f:
        f.write(session_id)

    st = time.time()

    response = await frame_agent.ainvoke(
        {"messages": [{"role": "user", "content": prompt}]}
    )

    os.makedirs(f"sessions/{session_id}", exist_ok=True)
    with open(f"sessions/{session_id}/frame_agent_process_addition.txt", "w", encoding="utf-8") as f:    
        for m in response["messages"]:
            f.write(m.pretty_repr() + "\n")
    
    et = time.time() - st
    print(f"Total time: {et} seconds")

    
    # After adding the rule, we can verify it's in the database
    conn = sqlite3.connect('database.db')
    cursor = conn.cursor()
    cursor.execute("SELECT * FROM process_rules WHERE process_rule_id = ?", (tour,))
    row = cursor.fetchone()
    if row is not None:
        rule_added = True
    else:
        rule_added = False
    conn.close()

    with open(f"sessions/{session_id}/frame_metadata.csv", "w", encoding="utf-8") as f:
        f.write(f"total_time_seconds,rule_added\n")
        f.write(f"{et},{rule_added}\n")
    
    # Run process rule

    print(f"Running process rule for session {session_id}...")
    
    process_prompt = f"""Run process_rule with process_rule_id '{tour}'."""

    st = time.time()

    response = await frame_agent.ainvoke(
        {"messages": [{"role": "user", "content": process_prompt}]}
    )

    os.makedirs(f"sessions/{session_id}", exist_ok=True)
    with open(f"sessions/{session_id}/frame_agent_process_execution.txt", "w", encoding="utf-8") as f:
        for m in response["messages"]:
            f.write(m.pretty_repr() + "\n")
    
    et = time.time() - st
    print(f"Total time: {et} seconds")

Adding process rule for session processadaptation_500_values_processadaptationmethod_generate_bpmn_tour_J09D_seed_409_exception_handling_db_error...
Total time: 16.34939694404602 seconds
Running process rule for session processadaptation_500_values_processadaptationmethod_generate_bpmn_tour_J09D_seed_409_exception_handling_db_error...
Total time: 33.62889385223389 seconds
Adding process rule for session processadaptation_0_values_processadaptationmethod_generate_bpmn_tour_J09D_seed_42_exception_handling_db_error...
Total time: 10.940216064453125 seconds
Running process rule for session processadaptation_0_values_processadaptationmethod_generate_bpmn_tour_J09D_seed_42_exception_handling_db_error...
Total time: 29.009209156036377 seconds
Adding process rule for session processadaptation_0_values_processadaptationmethod_generate_bpmn_tour_J09B_seed_1824_exception_handling_db_error...
Total time: 13.37000823020935 seconds
Running process rule for session processadaptation_0_values_processa

In [8]:
process_results_columns = ["session_id", "process_adaptation", "rule_adaptation_method", "tour", "seed", "all_correct"]
process_results = []

for run in runs:
    process_adaptation, rule_adaptation_method, tour, seed = run
    
    session_id = f"processadaptation_{process_adaptation}_processadaptationmethod_{rule_adaptation_method}_tour_{tour}_seed_{seed}_exception_handling_db_error"
    print(f"Session ID: {session_id}")

    support_email_found = False

    try:
        df_mails = pd.read_csv(f"sessions/{session_id}/process_metadata.csv")
        if "support@company.com" in df_mails["email_address"].values:
            support_email_found = True
    except Exception as e:
        print(f"Error processing session {session_id}: {e}")
    
    process_results.append([
        session_id,
        process_adaptation,
        rule_adaptation_method,
        tour,
        seed,
        support_email_found
    ])

df_results_add_rules = pd.DataFrame(process_results, columns=process_results_columns)
df_results_add_rules.to_csv("process_adaptation_results_generate_from_bpmn_rules_exception_handling_db_error.csv", index=False)
display(df_results_add_rules)
display(df_results_add_rules[df_results_add_rules["all_correct"] == False])

Session ID: processadaptation_500_values_processadaptationmethod_generate_bpmn_tour_J09D_seed_409_exception_handling_db_error
Session ID: processadaptation_0_values_processadaptationmethod_generate_bpmn_tour_J09D_seed_42_exception_handling_db_error
Session ID: processadaptation_0_values_processadaptationmethod_generate_bpmn_tour_J09B_seed_1824_exception_handling_db_error
Session ID: processadaptation_0_values_processadaptationmethod_generate_bpmn_tour_J09A_seed_409_exception_handling_db_error
Session ID: processadaptation_extension_estimates_processadaptationmethod_generate_bpmn_tour_J09B_seed_4506_exception_handling_db_error
Session ID: processadaptation_base_rule_processadaptationmethod_generate_bpmn_tour_J09B_seed_4506_exception_handling_db_error
Session ID: processadaptation_base_rule_processadaptationmethod_generate_bpmn_tour_J09B_seed_409_exception_handling_db_error
Session ID: processadaptation_0_values_processadaptationmethod_generate_bpmn_tour_J09A_seed_4506_exception_handling

,session_id,process_adaptation,rule_adaptation_method,tour,seed,all_correct
0,processadaptation_500_values_processadaptation...,500_values,generate_bpmn,J09D,409,True
1,processadaptation_0_values_processadaptationme...,0_values,generate_bpmn,J09D,42,True
2,processadaptation_0_values_processadaptationme...,0_values,generate_bpmn,J09B,1824,True
3,processadaptation_0_values_processadaptationme...,0_values,generate_bpmn,J09A,409,True
4,processadaptation_extension_estimates_processa...,extension_estimates,generate_bpmn,J09B,4506,True
5,processadaptation_base_rule_processadaptationm...,base_rule,generate_bpmn,J09B,4506,True
6,processadaptation_base_rule_processadaptationm...,base_rule,generate_bpmn,J09B,409,True
7,processadaptation_0_values_processadaptationme...,0_values,generate_bpmn,J09A,4506,True
8,processadaptation_500_values_processadaptation...,500_values,generate_bpmn,J09D,42,True
9,processadaptation_500_values_processadaptation...,500_values,generate_bpmn,J09D,4012,True


,session_id,process_adaptation,rule_adaptation_method,tour,seed,all_correct
